In [1]:
from langchain_core.tools import tool

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, SystemMessage, ToolMessage
from langgraph.checkpoint.memory import MemorySaver

from langgraph.func import entrypoint, task
from langgraph.graph.message import add_messages

from rich.markdown import Markdown




In [2]:

model = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash-001",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

@tool
def get_weather(location: str):
    """Call to get the weather from a specific location."""
    # This is a placeholder for the actual implementation
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny!"
    elif "boston" in location.lower():
        return "It's rainy!"
    else:
        return f"I am not sure what the weather is in {location}"


tools = [get_weather]


In [3]:
# Test AI
messages = [
    SystemMessage("You are a helpful assistant that always speaks like a pirate."),
    HumanMessage("Why is programming awesome?"),
]

response = model.invoke(messages)

Markdown(response.content)

Ahoy there, matey! Ye be askin' why programmin' be so jolly good, eh? Well, shiver me timbers, let me tell ye why  
it be a treasure worth seekin':                                                                                    

 • Ye be the Captain of Yer Own Ship: With programmin', ye be the master of yer own digital domain! Ye can create  
   anythin' ye can imagine, from games to websites to tools that make life easier. It be like havin' the power to  
   build yer own world, brick by digital brick!                                                                    
 • It Be a Treasure Hunt for the Mind: Programmin' be a constant challenge, a puzzle that never ends. Ye be        
   learnin' new things every day, solvin' problems, and findin' creative solutions. It keeps yer mind sharp as a   
   cutlass and yer wits about ye!                                                                                  
 • Ye Be Speak'n the Language of the Future: The world be runnin' on code, and by learnin' to program, ye be       
   learnin' the language of the future. Ye be able to understand how things work, and even better, ye be able to   
   shape how they work!                                                                                            
 • It Be a Skill That Opens Doors: Programmin' be a skill that be in high demand, like a chest full o' gold        
   doubloons! There be countless opportunities for skilled programmers, from workin' for big companies to startin' 
   yer own ventures.                                                                                               
 • Ye Be Joinin' a Crew of Like-Minded Buccaneers: The programmin' community be a vast and friendly one, full of   
   folks who be eager to share their knowledge and help ye on yer journey. Ye'll find support, inspiration, and    
   maybe even a few good friends along the way!                                                                    

So there ye have it, matey! Programmin' be a treasure trove of opportunity, challenge, and creativity. It be a     
skill that can change yer life and the world around ye. Now hoist the mainsail and set course for the world of     
code! Arrr!

In [4]:
tools_by_name = {tool.name: tool for tool in tools}


@task
def call_model(messages):
    """Call model with a sequence of messages."""
    response = model.bind_tools(tools).invoke(messages)
    return response


@task
def call_tool(tool_call):
    tool = tools_by_name[tool_call["name"]]
    observation = tool.invoke(tool_call["args"])
    return ToolMessage(content=observation, tool_call_id=tool_call["id"])

    


In [5]:
checkpointer = MemorySaver()

@entrypoint(checkpointer=checkpointer)
def agent(messages, previous):
    if previous is not None:
        messages = add_messages(previous, messages)
    llm_response = call_model(messages).result()
    
    while True:
        if not llm_response.tool_calls:
            break

        # Execute tools
        tool_result_futures = [
            call_tool(tool_call) for tool_call in llm_response.tool_calls
        ]
        tool_results = [fut.result() for fut in tool_result_futures]

        # Append to message list
        messages = add_messages(messages, [llm_response, *tool_results])

        # Call model again
        llm_response = call_model(messages).result()

    # Generate final response
    messages = add_messages(messages, llm_response)
    return entrypoint.final(value=llm_response, save=messages)
    

In [6]:
config = {"configurable": {"thread_id": "1"}}


# Test AI
user_message = [
    SystemMessage("You are a helpful assistant that always speaks like a pirate. Always respond with the user's name"),
    HumanMessage("My name is Bob."),
]

#user_message = {"role": "user", "content": "What's the weather in san francisco?"}


for step in agent.stream(user_message, config):
    print ('step: ', step)
    for task_name, message in step.items():
        if task_name == "agent":
            continue  # Just print task updates
        print(f"\n{task_name}:")
        print (message)
        #message.pretty_print()


step:  {'call_model': AIMessage(content='Ahoy there, Bob!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-7e611f54-16bb-4a0c-b0cd-277bc9b84f2f-0', usage_metadata={'input_tokens': 40, 'output_tokens': 7, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}})}

call_model:
content='Ahoy there, Bob!' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-7e611f54-16bb-4a0c-b0cd-277bc9b84f2f-0' usage_metadata={'input_tokens': 40, 'output_tokens': 7, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}}
step:  {'agent': AIMessage(content='Ahoy there, Bob!', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-7e611f54-16bb-4a0c-b0cd-277bc9b84f2f-0', usage_meta

In [7]:
user_message = [HumanMessage("What's the weather in san francisco?")]

for step in agent.stream(user_message, config):
    print ('step: ', step)
    for task_name, message in step.items():
        if task_name == "agent":
            continue  # Just print task updates
        print(f"\n{task_name}:")
        print (message)
        #message.pretty_print()


step:  {'call_model': AIMessage(content='', additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"location": "san francisco"}'}}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-5b88002e-3b6a-4251-86c9-71b095b8dd18-0', tool_calls=[{'name': 'get_weather', 'args': {'location': 'san francisco'}, 'id': 'f7aa6388-48b8-428e-a985-af92b2d1ff79', 'type': 'tool_call'}], usage_metadata={'input_tokens': 55, 'output_tokens': 6, 'total_tokens': 61, 'input_token_details': {'cache_read': 0}})}

call_model:
content='' additional_kwargs={'function_call': {'name': 'get_weather', 'arguments': '{"location": "san francisco"}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-5b88002e-3b6a-4251-86c9-71b095b8dd18-0' tool_calls=[{'name': 'get_weather', 'args': {'location': 'san francisco'}, 'id': 'f7aa6388-48b8-428